# Overlapping Generation Model (OLG) with Unfunded and Fully Funded Social Security System

Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm
import matplotlib.pyplot as plt


# Model description of OLG

The model we consider is based on notes from macroeconomics III. We examine the OLG-model where we first analyze an unfunded Pay-As-You-Go (PAYG) security system and how the tax rate affects the capital accumulation. We compare the unfunded PAYG-system with a fully funded PAYG-system. We solve the model analytically and numerically. We visualize the capital accumulation for the two models.  

# Model setup


We consider the **OLG model** where individuals live for two periods i.e. $t=1,2$ and the population is constant. Identical competitive firms maximize their profits employing a Cobb-Douglas technology that combines labor, $L_t$, and capital, $K_t$, so that $Y_t=A K_t^\alpha L_t^{1-\alpha}$ with $\alpha \in (0,1)$. We assume full capital depreciation. Under these assumptions, profit maximization leads to:

$$R = \alpha A k_t^{\alpha-1}$$

$$w_t = (1-\alpha)A k_t^{\alpha}$$

where $R$ is the rental rate of capital, $w_t$ is the wage rate, and $k_t$ denotes
capital in per-worker units. Utility for young individuals born in period $t$ is

$$U_t = \text{ln}(c_{1t}) + \frac{1}{1+\rho}\text{ln}(c_{2t+1})$$

with $\rho >-1$. The subscript $c_{1t}$ denotes consumption when young, $c_{2t+1}$ consumption when old. Young agents spend their entire time endowment, which is normalized
to one, working. Suppose the government runs an unfunded (pay-as-you-go)
social security system, according to which the young pay a contribution $d_t = \tau w_t$ that amounts to a fraction $\tau \in (0,1)$ of their wages. Thus, the contributions are paid
out in the same period to the current old. The latter do not work, and sustain their consumption though their savings and the social security benefits. Thus,
the budget constraints in each period of life read as:

$$c_{1t} + s_t = (1 -\tau ) w_t$$

$$c_{2t+1} = s_t R + d_{t+1} \qquad \vee \qquad c_{2t+1} = s_t R + \tau w_t$$ 



## Optimization

We begin by deriving the Euler equation and define lifetime utility:

In [2]:
# we set up symbols for the problem
c1 = sm.symbols('c_1t')
c2 = sm.symbols('c_2t+1')
rho = sm.symbols('rho')
w = sm.symbols('w_t')
s = sm.symbols('s_t')
R = sm.symbols('R')
tau = sm.symbols('tau')
lamb= sm.symbols('lambda')
k1 = sm.symbols('k_t+1')
k = sm.symbols('k_t')
A = sm.symbols('A')
alpha = sm.symbols('alpha')
kstar = sm.symbols('k^*')

# Lifetime utility
uc1 = sm.ln(c1)
uc2 = sm.ln(c2)
U = uc1 + 1/(1+rho) * uc2
display(U)

log(c_1t) + log(c_2t+1)/(rho + 1)

Defining the budget constraints:

In [3]:
#Budget constraints
con1 = (1-tau)*w-s
con2 = s*R + tau*w

#Lifetime budget constraint
bc = c1+((c2-tau*w)/R)-(w*(1-tau))
bc

c_1t - w_t*(1 - tau) + (c_2t+1 - tau*w_t)/R

Next, we set up the Lagrangian and the two FOC's:

In [4]:
#Lagrangian
L = U + lamb * (-1)*bc
display(L)

#FOC's
FOC1 = sm.Eq(0,sm.diff(L,c1))
FOC2 = sm.Eq(0,sm.diff(L,c2))
display(FOC1, FOC2)

-lambda*(c_1t - w_t*(1 - tau) + (c_2t+1 - tau*w_t)/R) + log(c_1t) + log(c_2t+1)/(rho + 1)

Eq(0, -lambda + 1/c_1t)

Eq(0, 1/(c_2t+1*(rho + 1)) - lambda/R)

We solve the FOC's with respect to lambda, so we can solve for the Euler equation:

In [5]:
lamb1 = sm.solve(FOC1,lamb)[0] 
lamb2 = sm.solve(FOC2,lamb)[0] 
display(lamb1, lamb2)

1/c_1t

R/(c_2t+1*(rho + 1))

Finally, we derive the Euler equation:

In [6]:
# Set the lambdas equal to eachother and solve for consumption when young 
euler1 = sm.solve(sm.Eq(lamb1,lamb2),c1)[0]
euler2 = sm.Eq(euler1, c1)
display(euler2)

Eq(c_2t+1*(rho + 1)/R, c_1t)

The Euler equation characterizes the slope of the optimal consumption path. The interpretation is that the marginal rate of substitution between current consumption $c_1$ and future consumption $c_{2t+1}$ is equated to the relative price between goods given by $R$ and the degree of patience $\rho$. 

Now we find the individual saving behaviour. We do this by substituting the Euler equation into the two budget constraints:

In [7]:
euler3 = euler2.subs(c1,con1) #substituting con1 with budget constraint for young
euler4 = euler3.subs(c2,con2) #substituting con2 with budget constraint for old
display(euler4)

Eq((rho + 1)*(R*s_t + tau*w_t)/R, -s_t + w_t*(1 - tau))

In [8]:
print('The optimal individual saving behaviour is given by:')
saving = sm.solve(euler4,s)[0] #solving substituted euler equation wrt. savings
opt_saving = sm.Eq(saving, s) #equate with s
display(opt_saving)

The optimal individual saving behaviour is given by:


Eq(w_t*(-R*tau + R - rho*tau - tau)/(R*(rho + 2)), s_t)

## Law-of-motion

The law of motion for capital is given by $k_{t+1} = s_t$ since only the young generation saves and the population $n$ is constant. We derive the capital accumulation equation $k_{t+1}$:

Substitute for the wage- and interest rate from the model setup and use aggregate capital stock to obtain the law-of-motion.

In [9]:
#Replace wage rate in optimal savings
savings = opt_saving.subs(w, ((1-alpha)*A*(k**(alpha))))

#Replace interest rate in optimal savings
savings_new = savings.subs(R, alpha*A*k**(alpha-1))

#Using aggregate capital
capital = k1
agcap = savings_new.subs(s,capital)

#Solving for k_t+1 to obtain law of-motion
lom = sm.solve(agcap,k1)[0]
lom = sm.Eq(k1,lom)
display(lom)

Eq(k_t+1, (A*alpha**2*k_t**alpha*tau - A*alpha**2*k_t**alpha - A*alpha*k_t**alpha*tau + A*alpha*k_t**alpha + alpha*k_t*rho*tau + alpha*k_t*tau - k_t*rho*tau - k_t*tau)/(alpha*(rho + 2)))

Use in steady state (SS) that $k_t = k_{t+1} = k^*$. The law-of-motion is: 

In [10]:
#Replace with k*
lom1 = lom.subs(k1, kstar)
lom2 = lom1.subs(k, kstar)

#Solve for k*
lom3 = sm.solve(lom2, kstar)[0]
display(lom3)

(A*alpha*(alpha*tau - alpha - tau + 1)/(-alpha*rho*tau + alpha*rho - alpha*tau + 2*alpha + rho*tau + tau))**(-1/(alpha - 1))

We choose relevant values of the parameters in the model. Then we compute the SS value by turning the law-of-motion into a python function:

In [11]:
#turn into python function
ss_func = sm.lambdify((rho, alpha, tau, A), lom3) 
ss_capital = ss_func(0.5, 1/3, 0.20, 15) #give the funtion random values
print(f'The Steady State level of capital for rho=0.5, alpha=1/3, tau=0.2, A=15 is {(ss_capital):.4f}')

The Steady State level of capital for rho=0.5, alpha=1/3, tau=0.2, A=15 is 4.1456


## Numerical solution

We solve the model numerically by a **bisection method**

In [12]:
#Parameters
alpha = 1/3 
rho = 0.5
tau = 0.20
A = 15
 
#Define objective function i.e. the law-of-motion as seen above
f = lambda k: k**alpha #input for the objective function
obj_kss = lambda kss: kss - ((A*alpha*f(kss)*(alpha*tau-alpha-tau+1))+tau*kss*(alpha*rho+alpha-rho-1))/(alpha*(rho+2))
#Optimizing using the bisect method
result = optimize.root_scalar(obj_kss, bracket=[0.01,100], method = 'bisect')

print(f'The Steady State level of capital for tau = 0.20, rho =0.5 , A = 15 and alpha = 1/3 is {(result.root):.4f}')    

The Steady State level of capital for tau = 0.20, rho =0.5 , A = 15 and alpha = 1/3 is 4.1456


This result shows that the numerical and analytical solution is identical. 

# Figures and visualization

We plot the capital accumulation equation and show how the taxrate $\tau$ affects the Steady State value. 

In [13]:
# Function that returns capital in the next period
def capital_accumulation(k,alpha,rho,A,tau):
    """ This function returns capital in period t+1
    Args:
        k (float): capital in period t
        alpha (float): cobb-douglas parameter
        rho (float): discount factor
        A (float): technology
        tau(float): tax rate
    Returns:
        value of capital in period t+1
    """
    k1 = ((A*alpha*(k**alpha)*(alpha*tau-alpha-tau+1))+tau*k*(alpha*rho+alpha-rho-1))/(alpha*(rho+2))
    
    return k1

# Function that return pairs of capital in period t and t+1
def transition(alpha,rho,A,tau):
    """ This function returns pairs of capital in period t and t+1
    Args:
        alpha (float): cobb-douglas parameter
        rho (float): discount factor
        A (float): technology
        tau (float): tax rate
    Returns:
        lists of values of capital in period t and t+1
    """
    k_nextperiod = [] # Values of capital in period t and t+1
    k_t = np.linspace(0,10,1000) # Range of capital in period t for the capital accumulation function to use

    for k in k_t: # Loop over different values of capital in period t
        value = capital_accumulation(k,alpha,rho,A,tau)
        k_nextperiod.append(value) # Appends values to the list
    
    return k_t,k_nextperiod
    
k_t,k_nextperiod = transition(alpha,rho,A,tau)
    
def fig(tau):
    """
    Returns:
    Plots the transition curve
    
    Args:
    tau(float): tax rate
    """
    #parameters
    alpha = 1/3
    tau = tau
    rho = 0.5
    A = 15
    
    #transition curve
    k_t, k_nextperiod = transition(alpha,rho,A,tau)
    
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(1,1,1)
    ax.set_title('Figure 1. Capital Accumulation and Steady State in Unfunded PAYG-system', fontsize=14)
    ax.plot(k_t,k_nextperiod, label="Transition curve") #transition curve
    ax.plot(k_t,k_t, '--', color='black',label="45 degree") #45 degree line
    ax.set_xlabel('$k_t$', fontsize=20)
    ax.set_ylabel('$k_{t+1}$', fontsize=20)
    ax.legend(fontsize=14)
    ax.grid()
    ax.set_xlim([0,10])
    ax.set_ylim([0,10]);
    return

import ipywidgets as widgets
widgets.interact(fig, 
    tau = widgets.FloatSlider(description=r'${\tau}$', min=0, max=1, step=0.05, value=0.2),
);

interactive(children=(FloatSlider(value=0.2, description='${\\tau}$', max=1.0, step=0.05), Output()), _dom_cla…

For a tax rate given by $\tau = 0.20$ the SS value is 4.1456. This is the default value of Figure 1. The Steady State value of capital is identical to the numerical and analytical solutions. For a higher value of $\tau$ we observe that the the SS-value of capital is decreasing. This is due to the fact that the young save less in order to maintain their consumption. 

# Model Extension

We impose a fully funded PAYG social security system in the model. The government raises taxes, $\tau$, from the young in period t and these contributions are paid out in the next period, together with the accrued interest rate. Budget constraints are given by:

$$ c_{1t} + s_t  = w_t(1-\tau) $$

$$ c_{2t+1} = R (s_t + \tau w_t) \qquad \text{for} \ t \geq T.$$

In [14]:
# we set up symbols for the problem
c1 = sm.symbols('c_1t')
c2 = sm.symbols('c_2t+1')
rho = sm.symbols('rho')
w = sm.symbols('w_t')
s = sm.symbols('s_t')
R = sm.symbols('R')
tau = sm.symbols('tau')
lamb= sm.symbols('lambda')
k1 = sm.symbols('k_t+1')
k = sm.symbols('k_t')
A = sm.symbols('A')
alpha = sm.symbols('alpha')
kstar = sm.symbols('k^*')

#Lifetime budget constraint
bc_1 = c1+(c2/R)-(tau*w)-w*(1-tau)
bc_1

c_1t - tau*w_t - w_t*(1 - tau) + c_2t+1/R

In [15]:
#lagrangian
L1 = U + lamb * (-1)*bc_1
L1

-lambda*(c_1t - tau*w_t - w_t*(1 - tau) + c_2t+1/R) + log(c_1t) + log(c_2t+1)/(rho + 1)

In [16]:
#First order conditions
FOC3 = sm.Eq(0,sm.diff(L1,c1))
FOC4 = sm.Eq(0,sm.diff(L1,c2))
display(FOC3)
display(FOC4)

Eq(0, -lambda + 1/c_1t)

Eq(0, 1/(c_2t+1*(rho + 1)) - lambda/R)

In [17]:
#Solving the FOC's
lamb1 = sm.solve(FOC3,lamb)[0]
lamb2 = sm.solve(FOC4,lamb)[0]
display(lamb1)
display(lamb2)

1/c_1t

R/(c_2t+1*(rho + 1))

In [18]:
#Solve for Euler equation by setting lambdas equal to eachother
euler1 = sm.solve(sm.Eq(lamb1,lamb2),c1)[0] 
euler2 = sm.Eq(euler1, c1)
display(euler2)

Eq(c_2t+1*(rho + 1)/R, c_1t)

The Euler equation has the same interpretation as before.

Now we find the individual saving behaviour. We do this by substituting the Euler equation into the two budget constraints:

In [19]:
euler3 = euler2.subs(c1,con1) #substituting con1 with budget constraint for young
euler4 = euler3.subs(c2,(s+w*tau)*R) #substituting con2 with budget constraint for old
display(euler4)

Eq((rho + 1)*(s_t + tau*w_t), -s_t + w_t*(1 - tau))

In [20]:
print('The optimal individual saving behaviour is now given by:')
saving = sm.solve(euler4,s)[0] #solving substituted euler equation wrt. savings
opt_saving = sm.Eq(saving, s) #equate with s
display(opt_saving)

The optimal individual saving behaviour is now given by:


Eq(w_t*(-rho*tau - 2*tau + 1)/(rho + 2), s_t)

## Law-of-motion

The law of motion for capital is still given by $k_{t+1} = s_t$ since only the young generation saves and the population is constant. We derive the capital accumulation equation $k_{t+1}$:

Substitute for the wage rate and use aggregate capital stock to obtain the **law-of-motion**.

In [21]:
#Replace wage rate in optimal savings
savings = opt_saving.subs(w, ((1-alpha)*A*(k**(alpha))))

#Using aggregate capital
capital = k1
agcap = savings.subs(s,capital)

#Solving for k_t+1 to obtain law of-motion
lom = sm.solve(agcap,k1)[0]
lom = sm.Eq(k1,lom)
display(lom)

Eq(k_t+1, A*k_t**alpha*(alpha*rho*tau + 2*alpha*tau - alpha - rho*tau - 2*tau + 1)/(rho + 2))

Use in steady state (SS) that $k_t = k_{t+1} = k^*$. The law-of-motion is: 

In [22]:
#Replace with k*
lom1 = lom.subs(k1, kstar)
lom2 = lom1.subs(k, kstar)

#Solve for k*
lom3 = sm.solve(lom2, kstar)[0]
display(lom3)

(A*(alpha*rho*tau + 2*alpha*tau - alpha - rho*tau - 2*tau + 1)/(rho + 2))**(-1/(alpha - 1))

We compute the SS value by turning the law-of-motion into a **python function**:

In [23]:
#turn into python function
ss_func_FF = sm.lambdify((rho, alpha, tau, A), lom3) 
ss_capital_FF = ss_func_FF(0.5, 1/3, 0.20, 15) #give the funtion random values
print(f'The Steady State level of capital for rho=0.5, alpha=1/3, tau=0.2, A=15 is {(ss_capital_FF):.4f}')

The Steady State level of capital for rho=0.5, alpha=1/3, tau=0.2, A=15 is 2.8284


## Numerical solution

We solve the model numerically by a **bisection method**

In [24]:
#Parameters
alpha = 1/3 
rho = 0.5
tau = 0.20
A = 15
 
#Define objective function i.e. the law-of-motion as seen above
f = lambda k: k**alpha #input for the objective function
obj_kss = lambda kss: kss - ((A*f(kss))*(alpha*rho*tau-alpha+2*alpha*tau-rho*tau-2*tau+1))/(rho+2)
#Optimizing using the bisect method
result = optimize.root_scalar(obj_kss, bracket=[0.01,100], method = 'bisect')

print(f'The Steady State level of capital for tau = 0.20, rho =0.5 , A = 15 and alpha = 1/3 is {(result.root):.4f}')    

The Steady State level of capital for tau = 0.20, rho =0.5 , A = 15 and alpha = 1/3 is 2.8284


This result shows that the numerical and analytical solution is identical. 

# Figures and visualization

We plot the capital accumulation equation and show how the taxrate $\tau$ affects the Steady State value. 

In [25]:
#Function that returns capital in the next period
def capital_accumulation(k,alpha,rho,A,tau):
    """ This function returns capital in period t+1
    Args:
        k (float): capital in period t
        alpha (float): cobb-douglas parameter
        rho (float): discount factor
        A (float): technology
        Tau(float): tax rate
    Returns:
        value of capital in period t+1
    """
    k1 = ((A*k**alpha)*(alpha*rho*tau-alpha+2*alpha*tau-rho*tau-2*tau+1))/(rho+2)
    
    return k1

#Function that return pairs of capital in period t and t+1
def transition(alpha,rho,A,tau):
    """ This function returns pairs of capital in period t and t+1
    Args:
        alpha (float): cobb-douglas parameter
        rho (float): discount factor
        A (float): technology
        tau (float): tax rate
    Returns:
        lists of values of capital in period t and t+1
    """
    k_nextperiod = [] # Values of capital in period t and t+1
    k_t = np.linspace(0,10,1000) # Range of capital in period t for the capital accumulation function to use

    for k in k_t: # Loop over different values of capital in period t
        value = capital_accumulation(k,alpha,rho,A,tau)
        k_nextperiod.append(value) # Appends values to the list
    
    return k_t,k_nextperiod
    
k_t,k_nextperiod = transition(alpha,rho,A,tau)
    
def fig(tau):
    """
    Returns:
    Plots the transition curve
    
    Args:
    tau(float): tax rate
    """
    #parameters
    alpha = 1/3
    tau = tau
    rho = 0.5
    A = 15
    
    #transition curve
    k_t, k_nextperiod = transition(alpha,rho,A,tau)
    
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(1,1,1)
    ax.set_title('Figure 2. Capital Accumulation and Steady State in Fully Funded PAYG-system', fontsize=14)
    ax.plot(k_t,k_nextperiod, color='red', label="Transition curve") #transition curve
    ax.plot(k_t,k_t, '--', color='black',label="45 degree") #45 degree line
    ax.set_xlabel('$k_t$', fontsize=20)
    ax.set_ylabel('$k_{t+1}$', fontsize=20)
    ax.legend(fontsize=14)
    ax.grid()
    ax.set_xlim([0,10])
    ax.set_ylim([0,10]);
    return

import ipywidgets as widgets
widgets.interact(fig, 
    tau = widgets.FloatSlider(description=r'${\tau}$', min=0, max=1, step=0.05, value=0.2),
);

interactive(children=(FloatSlider(value=0.2, description='${\\tau}$', max=1.0, step=0.05), Output()), _dom_cla…

We notice that the SS value of capital is lower in a fully funded PAYG-system even though the parameter values are the same. This is might due to the fact that individuals save less in a fully funded system compared to an unfunded system. We notice that the tax rate still affects the capital accumulation negatively, so higher taxrate will imply lower capital accumulation, ceteris paribus. The old generation finds itself with no pension thus they are worse off.    

# Conclusion

We analyze the OLG-model with a unfunded PAYG social security system. We solve the model numerically and analyze how different tax rates affects the capital accumulation and staeday state value. Higher tax rates imply lower capital accumulation and steady state values. We extend the model by consider a fully funded PAYG social security system. The only difference between the models is a lower steady state value of capital. The individuals tend to save less in a fully funded system.  